<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/C20_TARAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime

API_KEY = "API_KEY"

def get_top_coins(limit=10):
    url = f"https://min-api.cryptocompare.com/data/top/mktcapfull"
    headers = {'authorization': f'Apikey {API_KEY}'}
    params = {'limit': limit, 'tsym': 'USDT'}
    response = requests.get(url, params=params, headers=headers)
    data = response.json()['Data']
    return [coin['CoinInfo']['Name'] for coin in data]

def get_historical_data(symbol, timeframe='hour', limit=500):
    url = f"https://min-api.cryptocompare.com/data/v2/histo{timeframe}"
    headers = {'authorization': f'Apikey {API_KEY}'}
    params = {'fsym': symbol, 'tsym': 'USDT', 'limit': limit}
    response = requests.get(url, params=params, headers=headers)
    df = pd.DataFrame(response.json()['Data']['Data'])
    df['timestamp'] = pd.to_datetime(df['time'], unit='s')
    return df

def calculate_rsi(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)

    ma_up = up.ewm(com=period-1, adjust=True, min_periods=period).mean()
    ma_down = down.ewm(com=period-1, adjust=True, min_periods=period).mean()

    rsi = ma_up / ma_down
    rsi = 100 - (100 / (1 + rsi))
    return rsi

def calculate_c20_signals(data, symbol):
    data = data.copy()
    data['symbol'] = symbol
    data['logClose'] = np.log(data['close'])
    data['rsi'] = calculate_rsi(data['logClose'], 14)
    data['rsiChange'] = data['rsi'].diff()

    data['prev_rsiChange'] = data['rsiChange'].shift(1)
    data['c20_long'] = (data['rsiChange'].gt(20)) & (data['prev_rsiChange'].le(20))
    data['c20_short'] = (data['rsiChange'].lt(-20)) & (data['prev_rsiChange'].ge(-20))

    return data

def process_4h_data(df):
    df = df.set_index('timestamp')
    df_4h = df.resample('4H').agg({
        'time': 'first',
        'close': 'last',
        'high': 'max',
        'low': 'min',
        'open': 'first',
        'volumefrom': 'sum'
    }).dropna()
    return df_4h.reset_index()

def main():
    top_coins = get_top_coins(10)
    print(f"Analiz edilecek coinler: {', '.join(top_coins)}\n")

    for symbol in top_coins:
        print(f"\n{symbol} için sinyaller:")
        try:
            # 4H data
            df = get_historical_data(symbol, 'hour', 2000)
            df_4h = process_4h_data(df)
            signals_4h = calculate_c20_signals(df_4h, symbol)

            # Daily data
            df_daily = get_historical_data(symbol, 'day', 500)
            signals_daily = calculate_c20_signals(df_daily, symbol)

            # 4H sinyalleri
            signal_4h = signals_4h[signals_4h['c20_long'] | signals_4h['c20_short']]
            if not signal_4h.empty:
                print("\n4H Sinyaller:")
                signal_4h.apply(
                    lambda x: print(f"Tarih: {x['timestamp']}, "
                                f"Sinyal: {'LONG' if x['c20_long'] else 'SHORT'}, "
                                f"Fiyat: {x['close']:.2f}, "
                                f"RSI Değişim: {x['rsiChange']:.2f}"),
                    axis=1)

            # Günlük sinyaller
            signal_daily = signals_daily[signals_daily['c20_long'] | signals_daily['c20_short']]
            if not signal_daily.empty:
                print("\nGünlük Sinyaller:")
                signal_daily.apply(
                    lambda x: print(f"Tarih: {datetime.fromtimestamp(x['time'])}, "
                                f"Sinyal: {'LONG' if x['c20_long'] else 'SHORT'}, "
                                f"Fiyat: {x['close']:.2f}, "
                                f"RSI Değişim: {x['rsiChange']:.2f}"),
                    axis=1)

        except Exception as e:
            print(f"Hata: {symbol} için veri alınamadı - {str(e)}")
            continue

if __name__ == "__main__":
    main()

Analiz edilecek coinler: BTC, ETH, XRP, USDT, SOL, BNB, DOGE, ADA, SUI, USDC


BTC için sinyaller:


<ipython-input-1-f2c095c8ba0f>:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4h = df.resample('4H').agg({



4H Sinyaller:
Tarih: 2024-10-31 12:00:00, Sinyal: SHORT, Fiyat: 70620.26, RSI Değişim: -20.37
Tarih: 2024-11-06 00:00:00, Sinyal: LONG, Fiyat: 74343.96, RSI Değişim: 22.97

ETH için sinyaller:


<ipython-input-1-f2c095c8ba0f>:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4h = df.resample('4H').agg({



4H Sinyaller:
Tarih: 2024-11-06 00:00:00, Sinyal: LONG, Fiyat: 2579.36, RSI Değişim: 26.40
Tarih: 2024-12-20 12:00:00, Sinyal: LONG, Fiyat: 3374.68, RSI Değişim: 22.46

XRP için sinyaller:


...


4H Sinyaller:
Tarih: 2024-11-06 00:00:00, Sinyal: LONG, Fiyat: 0.36, RSI Değişim: 22.59
Tarih: 2024-12-20 12:00:00, Sinyal: LONG, Fiyat: 0.88, RSI Değişim: 20.82

SUI için sinyaller:


<ipython-input-1-f2c095c8ba0f>:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4h = df.resample('4H').agg({



4H Sinyaller:
Tarih: 2024-12-20 12:00:00, Sinyal: LONG, Fiyat: 4.29, RSI Değişim: 28.81

Günlük Sinyaller:
Tarih: 2024-08-08 00:00:00, Sinyal: LONG, Fiyat: 0.85, RSI Değişim: 22.38

USDC için sinyaller:


<ipython-input-1-f2c095c8ba0f>:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4h = df.resample('4H').agg({



4H Sinyaller:
Tarih: 2024-10-22 04:00:00, Sinyal: LONG, Fiyat: 1.00, RSI Değişim: 41.61
Tarih: 2024-10-22 12:00:00, Sinyal: SHORT, Fiyat: 1.00, RSI Değişim: -45.12
Tarih: 2024-10-24 16:00:00, Sinyal: SHORT, Fiyat: 1.00, RSI Değişim: -29.75
Tarih: 2024-10-24 20:00:00, Sinyal: LONG, Fiyat: 1.00, RSI Değişim: 20.83
Tarih: 2024-10-31 12:00:00, Sinyal: LONG, Fiyat: 1.00, RSI Değişim: 22.71
Tarih: 2024-11-01 12:00:00, Sinyal: SHORT, Fiyat: 1.00, RSI Değişim: -23.17
Tarih: 2024-11-04 00:00:00, Sinyal: LONG, Fiyat: 1.00, RSI Değişim: 21.64
Tarih: 2024-11-04 12:00:00, Sinyal: SHORT, Fiyat: 1.00, RSI Değişim: -20.55
Tarih: 2024-12-09 20:00:00, Sinyal: SHORT, Fiyat: 1.00, RSI Değişim: -22.46
Tarih: 2024-12-19 00:00:00, Sinyal: LONG, Fiyat: 1.00, RSI Değişim: 26.75
Tarih: 2024-12-20 12:00:00, Sinyal: SHORT, Fiyat: 1.00, RSI Değişim: -33.02
Tarih: 2024-12-24 08:00:00, Sinyal: SHORT, Fiyat: 1.00, RSI Değişim: -21.14
Tarih: 2024-12-27 16:00:00, Sinyal: LONG, Fiyat: 1.00, RSI Değişim: 20.84
Tarih: 20